In [ ]:
!wget https://github.com/pauldebayan/RockPaperScissor-Game/raw/refs/heads/main/rockPaperScissor_dataset.zip
!wget https://raw.githubusercontent.com/pauldebayan/RockPaperScissor-Game/refs/heads/main/labels.csv
!unzip rockPaperScissor_dataset.zip
!rm -rf rockPaperScissor_dataset.zip

In [109]:
from torchvision.io import read_image
import matplotlib.pyplot as plt
import torchvision
import torch
import torch.nn as nn
import os
import pandas as pd
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
img = read_image('./rockPaperScissor_dataset/rock1.jpg')

print(img.shape)

In [112]:
class RockPaperScissorDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)

        return image, label

In [113]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

In [114]:
dataset = RockPaperScissorDataset(img_dir = 'rockPaperScissor_dataset',
                             annotations_file = 'labels.csv',
                             transform = transform)

In [115]:
train_set, valid_set, test_set = torch.utils.data.random_split(dataset, [0.6, 0.2, 0.2])

In [116]:
train_dataloader = DataLoader(train_set, batch_size=5, shuffle=True)
valid_dataloader = DataLoader(valid_set, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=1, shuffle=True)

In [117]:
class RockPaperScissorCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 20, 3, stride = 1)
        self.conv2 = nn.Conv2d(20, 10, 3, stride = 1)
        self.conv3 = nn.Conv2d(10, 5, 3, stride = 1)

        self.pool = nn.MaxPool2d(2, 2)

        self.activ = nn.LeakyReLU()

        self.dropout = nn.Dropout(0.2)

        self.linear_stack = nn.Sequential(
            nn.Linear(6125, 50),
            nn.LeakyReLU(),
            nn.Linear(50, 10),
            nn.LeakyReLU(),
            nn.Linear(10, 3),
        )

    def forward(self, x):

        x = self.pool(self.activ(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(self.activ(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(self.activ(self.conv3(x)))

        x = x.view(x.size(0), -1)
        x = self.linear_stack(x)

        return x

In [ ]:
model = RockPaperScissorCNN().to(device)
print(model)

In [ ]:
print(img.shape)
model(img.float().to(device))

In [ ]:
# Display image and label.
train_features, train_labels = next(iter(train_dataloader))
img = train_features[0]
label = train_labels[0]

print(img.shape)

In [122]:
learning_rate = 0.0001
batch_size = 5
epochs = 30
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [123]:
def train_loop():

    size = len(train_dataloader)
    accumulate_loss = 0

    for img, label in train_dataloader:

        img = img.float().to(device)
        label = label.to(device)

        pred = model(img)

        loss = loss_fn(pred, label)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        accumulate_loss += loss.item()

    return accumulate_loss/size


def valid_loop():
    size = len(valid_dataloader)
    accumulate_loss = 0

    for img, label in valid_dataloader:

        img = img.float().to(device)
        label = label.to(device)

        pred = model(img)
        loss = loss_fn(pred, label)

        accumulate_loss += loss.item()


    return accumulate_loss/size



In [ ]:
x = [x+1 for x in range(epochs)]
trainLossArr = []
validLossArr = []

for i in range(epochs):

    # Train
    train_loss = train_loop()
    trainLossArr.append(train_loss)

    # Valid
    valid_loss = train_loop()
    validLossArr.append(valid_loss)

    print(f"Epoch {i+1} - Training Loss: {train_loss}, Validation Loss: {valid_loss}")


plt.plot(x, trainLossArr, 'b', label="Train")
plt.plot(x, validLossArr, 'g', label="Valid")
plt.legend(loc="upper right")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [125]:
torch.save(model.state_dict(), 'model.pt')

In [ ]:
state_dict = torch.load('model.pt', map_location=device, weights_only=True)
model.load_state_dict(state_dict)
model.eval()

In [ ]:
correct = 0
for img, label in test_dataloader:
    img = img.float().to(device)
    label = label.to(device)
    pred = model(img)

    if torch.argmax(pred) == label.squeeze():
        correct += 1
accuracy = correct/len(test_dataloader)*100
print(f"Accuracy: {accuracy}")


In [ ]:
!pip install onnx

In [137]:
import onnx

img = torch.randn(1, 3, 300, 300).to(device)

torch.onnx.export(model, img, "model.onnx", input_names=['input'])

In [ ]:
from google.colab import files
files.download('model.onnx')